# Undistort Dataset

In [2]:
from models.dataset.dataset_container import DatasetContainer
from tqdm import tqdm
from pathlib import Path

In [3]:
resource_path = Path("../../resources")

class Args:
    uncalibrated_dataset_path = resource_path / "images/uncalibrated/old/dataset_1.npz"
    calibrated_dataset_path = resource_path / "images/calibrated/dataset_1.npz"
args = Args()

## Actual Undistortion of all images

In [4]:
import cv2
import numpy as np
# Load Dataset
dataset_container = DatasetContainer()
dataset_container.load_from_dataset(args.uncalibrated_dataset_path)
print(f"Opened dataset containing {dataset_container.size()} image sets")

# initialize empty dataset container adopting both camera intrinsics
remapped_dataset_container = DatasetContainer()
remapped_dataset_container.load_from_intrinsics(*dataset_container.get_camera_intrinscs())

## Load Coefficients
# calibration = StereoCalibration()
# calibration.load_calibration(path=args.calibration_file_path)
calibration = np.loadtxt(resource_path / Path("calibrations/regression/old/calibration_from_1_2_3.np"))
# print(f"Calibration loaded: {args.calibration_file_path}")

## Undistort Images (iteration over dataset_container retrieves 4-tuples with corresponding images)
for rs_rgb, rs_depth, zv_rgb, zv_depth in tqdm(dataset_container, desc='remap tuples'):
    # rs_rgb, zv_rgb = calibration.remap(rs_rgb_img, zv_rgb_img)
    # rs_depth, zv_depth = calibration.remap(rs_depth_img, zv_depth_img)
    rs_rgb = cv2.warpPerspective(rs_rgb, calibration, (zv_rgb.shape[1], zv_rgb.shape[0]))
    rs_depth = cv2.warpPerspective(rs_depth, calibration, (zv_rgb.shape[1], zv_rgb.shape[0]))
    # add remapped images to according dataset container
    remapped_dataset_container.append(rs_rgb, rs_depth, zv_rgb, zv_depth)

print("Remapped all images")

## save image dataset
remapped_dataset_container.save(args.calibrated_dataset_path)

print(f"saved undistorted images in {args.calibrated_dataset_path}")

Opened dataset containing 24 image sets


remap tuples: 24it [00:00, 154.19it/s]

Remapped all images


saved undistorted images in ../../resources/images/calibrated/dataset_1.npz
